In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

<h1>Exploratory Analysis on Mass Shootings in the United States</h1>

Data provided by Kaggle.com.  I will be peforming an Exploratory Analysis on US Mass shooting to find any correlation between shooters and specific days of the year.

• How many people got killed and injured per year?

• Visualize mass shootings on US map

• Is there any correlation between shooter and his/her race, gender

• Any correlation with calendar dates? Do we have more deadly days, weeks or months on average

• What cities and states are more prone to such attacks

• Can you find and combine any other external dataset to enrich the analysis

• Any other pattern you see that can help in prediction, crowd safety or in-depth analysis of the event

• How many shooters have some kind of mental health problem? Can we compare that shooter with general population with same condition

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/Mass Shootings Dataset.csv', encoding = "ISO-8859-1")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.Summary = df.Summary.fillna('Stephen Paddock shot and from Mandalay Bay Hotel Room')
df.Gender = df.Gender.fillna('M')
df.Race = df.Race.fillna('White')

In [ ]:
df.isnull().sum()

In [ ]:
df.Gender.replace(['M', 'M/F'], ['Male', 'Male/Female'], inplace=True)

In [ ]:
t = pd.DataFrame(pd.Series(df['Race'].sort_values(ascending=True).unique()))
v = pd.DataFrame(pd.Series(['Asian', 'Asian', 'Asian','Black','Black','Black','Latino','Native American',
           'Native American', 'Other', 'Other', 'Other', 'Unknown', 'White', 'White', 
           'White', 'White', 'Black', 'Unknown', 'White']))
t.reset_index(inplace=True)
v.reset_index(inplace=True)
u = pd.merge(t,v,how='inner',on='index')

In [ ]:
mydict = dict(zip(u['0_x'], u['0_y']))

In [ ]:

for i in df.columns:
    if i == 'Race':
        df['Race'] = df[i].map(mydict) 

In [ ]:
df.Date = pd.to_datetime(df.Date)

In [ ]:
df1 = df.drop(['Total victims', 'Longitude', 'Latitude', 'S#', 'Summary', 'Title'], axis=1)

The data has duplicates here is how I found the duplicates I have removed total victims since is not calcualted correctly, removed Longitude, Latitude this has diffirent locations summary has two different summaries to the same location, fatalies and injured.  Below you can see the duplicate rows after removing the the above features.   Then I perform a pandas dupicated fucntion to find those and then droping the duplicates this ensure that there is no dupicates in the data.  

In [ ]:
df1[df1['Location']=='Newtown, Connecticut']

In [ ]:
df1.duplicated()
df2 = df1.drop_duplicates()

Recalculate the Total victims values by adding the Fatalities and injured.  

In [ ]:
df2['Total victims'] = df2['Fatalities'] + df['Injured']
df2['Total victims'] = df2['Total victims'].astype(int)

In [ ]:
df2.head()

In [ ]:
df2['Year'] = df2['Date'].dt.year

In [ ]:
yrvictims = df2.groupby('Year').sum()['Injured']
count = df2.groupby('Year').count()['Location']
yrfatalities = df2.groupby('Year').sum()['Fatalities']


In [ ]:
e=pd.DataFrame(yrvictims)
e.reset_index(inplace=True)
e.shape

In [ ]:
c = pd.DataFrame(count)
c.reset_index(inplace=True)
c.shape


In [ ]:
t = pd.DataFrame(yrfatalities)
t.reset_index(inplace=True)
t.shape

In [ ]:
z = pd.merge(e,c,how='inner',on='Year')
w = pd.merge(z,t, how='inner', on='Year')

In [ ]:
w.head()

In [ ]:
n_groups = w.Year

fig, ax = plt.subplots(figsize=(15,7))

bar_width = 0.3

opacity = 0.4

rects1 = plt.bar(n_groups, w['Location'], bar_width ,
                 alpha=opacity,
                 color='b',
                 label='Number of Attacks')

rects2 = plt.bar(n_groups + bar_width, w['Fatalities'], bar_width,
                 alpha=opacity,
                 color='r',
                 label='Total Number of Fatalities')

rects3 = plt.bar(n_groups + bar_width  * 2, w['Injured'], bar_width,
                 alpha=opacity,
                 color='g',
                 label='Total Number of Injured')

plt.xlabel('Year')
plt.ylabel('Count')
plt.title('Total numbers of Attacks, Fatalities and Injured')

plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
df2['City'] = df2['Location'].str.rpartition(',')[0]
df2['State'] = df2['Location'].str.rpartition(',')[2]

In [ ]:
df2['Mental Health Issues'].replace(['unknown', 'unclear', 'Unclear'], ['Unknown','Unclear', 'Unclear'], inplace=True)

In [ ]:
df2.head()

The Below Scatter plot uses the Coordinate of where the mass shooting occur. We see from the plot and learned where this shooting occur and also which area can be consider a high risk area or an area prone for mass shooting.  What stands out is the pacific Northwest, Southern California, Mid Atlantic region and some East Coast areas.  One of the reasons why these area have a high number of attacks is because is heavily dense populated areas.  

In [ ]:
h = df.groupby(['Longitude', 'Latitude', 'Location']).count()['S#']
d = pd.DataFrame(h)
d.reset_index(inplace=True)
n = d['Location']
x = d['Longitude']
y = d['Latitude']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y, s=d['S#'] *75)
ax.set_ylabel('Longitude')
ax.set_xlabel('Latitude')
ax.set_title('Plotting Location by Number of Attacks', fontsize=18)
for i, txt in enumerate(n):
    ax.annotate(txt, (x[i], y[i]), fontsize=8)
plt.show()

Lets transform the features of the data and use LabelEncoder to get the data ready and use for corralation. Depending on what we find in the exploratory analysis of this dataset I will perform Machine Learning to determine what can be predicted from using this data.  

In [ ]:
df3 = df2.copy()

from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in df3.columns:
    df3[col] = labelencoder.fit_transform(df3[col])
df3.head()

In [ ]:
plt.figure(figsize=(14,8))
sns.heatmap(df3.corr(),annot=True)
plt.show()

From the above Corralation map we can see some of the feature are Corrolated.  For example Race and Fatality, Race and Injured, Race and Total Victim are corralated.  Another Corralation that stands out for me is Mental Health Issues which is coralted with Race, Total Victims, Injured.  Later on I will analyze the mental health Feature to determine if the attackers had mental issues or not.   

In [ ]:
w = df2['Mental Health Issues'].value_counts()


In [ ]:

plt.figure(figsize=(15,10))
sns.boxplot(df2['Race'], df2['Fatalities'])
plt.xlabel('Attackers Race')
plt.title('Count of Fatalities by Attacker', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(df2['Date'], df2['Injured'])
plt.plot(df2['Date'], df2['Fatalities'])
plt.title('Total by the Years', fontsize=24)
plt.ylabel('Number of Victims')
plt.xlabel('Year')
plt.show()

In [ ]:
df2[df2['Location']=='Newtown, Connecticut']

In [ ]:
plt.figure(figsize=(15,10))
b = df2.groupby('Mental Health Issues').sum()[['Fatalities', 'Injured', 'Total victims']]
sns.barplot(b.index, b.Fatalities)
plt.ylabel('Total Fatalities')
plt.title('Total Fatalities By Mental Health Illness', fontsize=20)
plt.show()

In [ ]:
b

In [ ]:
health = df2.groupby('Mental Health Issues').sum()['Fatalities']
health = pd.DataFrame(health)
health.reset_index(inplace=True)

plt.figure(figsize=(15,8))
plt.pie(health.Fatalities, labels=health['Mental Health Issues'], autopct='%1.1f%%', shadow=True)
plt.axis('equal')
plt.title('Percentage of Fatalities by Mental Health', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(df2['Mental Health Issues'], df2['Fatalities'])
plt.title('Total Fatalities by Mental Health Illness', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(df2['Mental Health Issues'], df['Total victims'])
plt.title('Total victims by Mental Health Illness', fontsize=18)
plt.show()

In [ ]:
countState = df2['State'].value_counts()
countState = countState.head(10)
plt.figure(figsize=(15,8))
sns.barplot(countState.index, countState.values)
plt.xticks(rotation='vertical')
plt.title('Total Attacks by State', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
plt.pie(countState.values, labels=countState.index, autopct='%1.1f%%', shadow=True)
plt.axis('equal')
plt.title('Attacks by top 10 States', fontsize=18)
plt.show()

In [ ]:
ga = df2.groupby('State').sum()['Fatalities']
ga = pd.DataFrame(ga.sort_values(ascending=False).head(10))

ga.reset_index(inplace=True)

plt.figure(figsize=(15,8))
plt.pie(ga.Fatalities, labels=ga.State, autopct='%1.1f%%', shadow=True)
plt.axis('equal')
plt.title('Percentage of Fatalitie by State', fontsize=18)
plt.show()

In [ ]:
cityCount = df2['City'].value_counts()
cityCount = cityCount.head(10)
plt.figure(figsize=(15,8))
sns.barplot(cityCount.index, cityCount.values)
plt.xticks(rotation='vertical')
plt.title('Total Attacks by Cities', fontsize=18)
plt.show()

In [ ]:
df2.head()

In [ ]:
raceCounts = df2.groupby('Race').sum()[['Fatalities','Injured', 'Total victims']]

In [ ]:
raceCounts.reset_index(inplace = True)

According to the data it is suggested that 55% fatalities were cause by White race attaker. 

In [ ]:
plt.figure(figsize=(15,8))
plt.pie(raceCounts.Fatalities, labels=raceCounts.Race, autopct='%1.1f%%', shadow=True)
plt.axis('equal')
plt.title('Percentage of Fatalitie Cause by Race of Attacker', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(raceCounts.Race, raceCounts.Fatalities)
plt.title('Fatalities by Attackers Race', fontsize=18)
plt.show()

In [ ]:
n_groups = np.arange(len(raceCounts.Race))

labels = np.array(raceCounts.Race)

fig, ax = plt.subplots(figsize=(15,7))

bar_width = 0.34

opacity = 0.4

rects1 = ax.bar(n_groups, raceCounts.Fatalities, bar_width ,
                 alpha=opacity,
                 color='b',
                 label='Number of Fatalities')

#rects2 = plt.bar(n_groups + bar_width, w['Fatalities'], bar_width,
                 #alpha=opacity,
                 #color='r',
                 #label='Total Number of Fatalities')

rects2 = ax.bar(n_groups + bar_width, raceCounts.Injured, bar_width,
                 alpha=opacity,
                 color='r',
                 label='Total Number of Injured')

ax.set_xlabel('Race')
ax.set_ylabel('Count')
plt.xticks(n_groups,labels)
ax.set_title('Total numbers of Fatalities and Injured by Attackers Race', fontsize=18)

ax.legend()

plt.tight_layout()
plt.show()